In [20]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
import datetime as dt
%matplotlib inline

In [2]:
def rmsle(y_true, y_pred):
    return np.sqrt(np.mean((np.log1p(y_true)-np.log1p(y_pred))**2))

In [3]:
train = pd.read_csv("train.zip", compression="zip")
test = pd.read_csv("test.csv")

In [4]:
# Используем среднее для предсказания
y_pred = np.array([train.trip_duration.mean()]*len(train))
rmsle(train.trip_duration, y_pred)

0.89084622891827581

In [5]:
# Используем медиану для предсказания
y_pred = np.array([train.trip_duration.median()]*len(train))
rmsle(train.trip_duration, y_pred)

0.79631689594871091

In [6]:
# Считаем расстояние и строим линейную регрессию на 1 признаке

In [7]:
train["dist"] = np.sqrt((train.pickup_longitude-train.dropoff_longitude)**2+(train.pickup_latitude-train.dropoff_latitude)**2)

In [8]:
X = train
y = train.trip_duration

In [25]:
# Строим признаки: часы и бинарный признак: выходной или нет

In [9]:
X.pickup_datetime = X.pickup_datetime.apply(lambda x: dt.datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
# X.dropoff_datetime = X.dropoff_datetime.apply(lambda x: dt.datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))

In [10]:
X["hour"] = X.pickup_datetime.apply(lambda x: x.hour)
X["is_weekend"] = X.pickup_datetime.apply(lambda x: 1 if x.weekday() in [5, 6] else 0)

In [11]:
X = pd.get_dummies(X, columns=["hour"])

In [17]:
cols = ["dist", "is_weekend"] + [x for x in X if "hour" in x]
len(cols)

26

In [24]:
model = RandomForestRegressor(n_estimators=100, max_depth=3, n_jobs=-1)
model.fit(X[cols], np.log1p(y))
rmsle(y, np.expm1(model.predict(X[cols])))

0.51345095797347673